In [ ]:
%%capture
!pip install pyxnat
!pip install "pyCERR[napari] @ git+https://github.com/cerr/pyCERR.git"
!pip install sevenbridges-python

In [ ]:
import os

In [ ]:
from pyxnat import Interface
from glob import glob
import shutil


def getXNATData(xhost,user,scandict,downloadDir):
  xnat = Interface(xhost, user)
  os.makedirs(downloadDir, exist_ok=True)
  expdirlist = []
  for scan_entry in scandict:
    proj = scan_entry['exam']['proj']
    subj = scan_entry['exam']['subj']
    exp = scan_entry['exam']['exp']
    scan_list = scan_entry['exam']['scan_list']
    expdir = os.path.join(downloadDir,exp)
    expdirlist.append(expdir)
    os.makedirs(expdir, exist_ok = True)
    xexp = xnat.select.project(proj).subject(subj).experiment(exp)
    for scan in scan_list:
      xres = xnat.select.project(proj).subject(subj).experiment(exp).scan(scan).resource('DICOM')
      if not xres.exists():
        xres = xnat.select.project(proj).subject(subj).experiment(exp).scan(scan).resource('secondary')
      xres.get(downloadDir,extract=True)
    for dcmfolder in ['DICOM','secondary']:
      dcmlist = glob(os.path.join(downloadDir,dcmfolder,'*.dcm'))
      print(dcmlist)
      for dcm in dcmlist:
        shutil.move(dcm, expdir)
  for dcmfolder in ['DICOM','secondary']:
    if os.path.exists(os.path.join(downloadDir,dcmfolder)):
      os.rmdir(os.path.join(downloadDir,dcmfolder))
    if os.path.exists(os.path.join(downloadDir,dcmfolder + '.zip')):
      os.remove(os.path.join(downloadDir,dcmfolder + '.zip'))
  xnat.disconnect()
  return expdirlist


In [ ]:
xhost = 'https://xnat.yoursite.org'
user = 'xusr'

#example of dictionary to describe the data you want to pull locally from XNAT,
# one entry per experiment, iterate over scans
scandict = [{'exam':
              {'proj':'project_id',
                'subj':'subj_id',
                'exp':'experiment_id',
                'scan_list':['1','2']
                }
            }]

dataPath = '/content/images'
os.makedirs(dataPath,exist_ok=True)

In [ ]:
folderList = getXNATData(xhost,user,scandict,dataPath)

In [ ]:
from cerr import plan_container as pc

In [ ]:
# Import DICOM to planC
planC = pc.loadDcmDir(folderList[0])

In [ ]:
from cerr.viewer import showMplNb
showMplNb(planC=planC, scan_nums=0, struct_nums = [0,1,2,3,4,5], windowCenter=-400, windowWidth=2000)